In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [69]:
records = pd.read_excel(
    "coffee_shop_sales.xlsx", 
    index_col="transaction_id", 
    #nrows=100
)

In [71]:
transaction_datetime = pd.to_datetime(records["transaction_date"].astype(str) + " " + records["transaction_time"].astype(str))
transaction_datetime = pd.Series(transaction_datetime, name="transaction_datetime",index=None)

records = records.reindex([
    "transaction_datetime", 'transaction_qty', 'store_id', 
    'store_location', 'product_id', 'unit_price', 
    'product_category', 'product_type', 'product_detail'], axis="columns")

records["transaction_datetime"] = transaction_datetime

records.sort_values("transaction_datetime", inplace=True)
records.head()

,transaction_datetime,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
transaction_id,,,,,,,,,
1,2023-01-01 07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
2,2023-01-01 07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
3,2023-01-01 07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
4,2023-01-01 07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
5,2023-01-01 07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [73]:
records["store_location"].value_counts()

store_location
Hell's Kitchen     50735
Astoria            50599
Lower Manhattan    47782
Name: count, dtype: int64

In [79]:
def get_revenue(df: pd.DataFrame, amount:str, unit_price:str) -> float:
    return (df[amount] * df[unit_price]).sum()

records.groupby("store_location").apply(get_revenue, amount="transaction_qty", unit_price="unit_price", include_groups=False)

store_location
Astoria            232243.91
Hell's Kitchen     236511.17
Lower Manhattan    230057.25
dtype: float64

In [99]:
(records
 .groupby("store_location")[["transaction_qty","unit_price"]]
 .apply(pd.DataFrame.describe, include_groups=False)
)

transaction_qty    unit_price
store_location                                      
Astoria         count     50599.000000  50599.000000
                mean          1.403012      3.384897
                std           0.490508      2.513269
                min           1.000000      0.800000
                25%           1.000000      2.500000
                50%           1.000000      3.000000
                75%           2.000000      3.750000
                max           2.000000     45.000000
Hell's Kitchen  count     50735.000000  50735.000000
                mean          1.413955      3.394218
                std           0.500879      2.777859
                min           1.000000      0.800000
                25%           1.000000      2.500000
                50%           1.000000      3.000000
                75%           2.000000      3.750000
                max           8.000000     45.000000
Lower Manhattan count     47782.000000  47782.000000
                mean          1.501444      3.366643
                std           0.625618      2.679107
                min           1.000000      0.800000
                25%           1.000000      2.500000
                50%           1.000000      3.000000
                75%           2.000000      3.750000
                max           6.000000     45.000000